### Assignment 8


This will produce polynomials U, V, W, and HT (see the notation in the book).

Do an encrypted evaluation of each of these polynomials, this will result in

$$
eval(U) = [A]_1\\eval(V)=[B]_2,\\ eval(W)=[C']_1,\\ eval(HT)=[HT]_1
$$

Create

$$
[C]=[C']_1+[HT]_1
$$

Then verify that
 
 
 
 
Do the verification on chain.
Your code should be able to start with an arbitrary R1CS and compute the three elliptic curve points
 
 
 
for which the verifier will check if the pairing is correct.
Optimization
It takes a long time for the galois library to instantiate the curve order GF(curve_order). You can speed up the process by doing

```python
from py_ecc.bn128 import curve_order
import galois
GF = galois.GF(curve_order, primitive_element=5, verify=False)
```

In [1]:
from py_ecc.bn128 import G1, G2, pairing, multiply, FQ, FQ2
from py_ecc.bn128.bn128_curve import b, b2
import numpy as np
import galois
import functools
import galois 
import numpy as np
from py_ecc.bn128 import curve_order
import sympy as sp

# Use the curve_order from py_ecc.bn128
GF = galois.GF(curve_order, primitive_element=5, verify=False)


# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick values for x and y
x = 100
y = 100

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3
# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])
print(A.shape)
print(B.shape)
print(C.shape)
print(w.shape)

(3, 6)
(3, 6)
(3, 6)
(6,)


In [2]:

def mat_to_finite_field(mat : np.array) -> np.array:
    return (mat + curve_order) % curve_order


In [3]:
from functools import partial
tao = 3



from typing import List, Tuple

def gen_points(tao: int, n_rows: int) -> Tuple[list, list]:
    arrs = []
    for point in [G1, G2]:
        arr = []
        for i in range(n_rows):
            arr.append(multiply(point, tao**i))
        arrs.append(arr)
    return tuple(arrs)



In [4]:
from py_ecc.fields import bn128_FQ2, bn128_FQ

def is_g2(a):
    if not (isinstance(a[0], bn128_FQ2) and isinstance(a[1], bn128_FQ2)):
        return False
    
    return True

def is_g1(a):
    if not (isinstance(a[0], bn128_FQ) and isinstance(a[1], bn128_FQ)):
        return False
    return True


In [26]:
from py_ecc.bn128 import G1, G2, pairing, multiply, add, curve_order, eq
#exercise 3


def eval_poly(coeffs: list[int], point: list):
    is_g1_point = is_g1(point[0])
    val = G1 if is_g1_point else G2
    
    for i, coeff in enumerate(coeffs):
        val = add(val, multiply(point[i], int(coeff)))
    
    return val

    
    

def mat_to_finite_field(mat : np.array) -> np.array:
    return (mat + curve_order) % curve_order

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return functools.reduce(sum_, map(mul_, polys, witness))


def poly_to_np(poly : galois.Poly, n_rows : int)-> np.array:
    
    elms = [int(coeff) for coeff in poly.coeffs]
    if len(elms) < n_rows:
        elms = elms + [0] * (n_rows - len(elms)) # pad with zeros if not enough elements
    return np.array(elms)

def qap_finite_field(A, B, C, w):
    n_rows = A.shape[0]
    
    tao = 5
    g1_points, g2_points = gen_points(tao, n_rows)
    
    def interpolate_column(col):
        xs = GF(np.arange(1, n_rows + 1))
        return galois.lagrange_poly(xs, col)

    A_polys = np.apply_along_axis(interpolate_column, 0, A)
    B_polys = np.apply_along_axis(interpolate_column, 0, B)
    C_polys = np.apply_along_axis(interpolate_column, 0, C)
    
    term1 = inner_product_polynomials_with_witness(A_polys, w)
    term2 = inner_product_polynomials_with_witness(B_polys, w)
    term3 = inner_product_polynomials_with_witness(C_polys, w)

    t = galois.Poly([1], field=GF)
    for i in range(1, n_rows + 1):
        t *= galois.Poly([1, curve_order - i], field=GF)

    h = (term1 * term2 - term3) // t
    
    poly_h = poly_to_np(h, n_rows)
    poly_A = poly_to_np(term1, n_rows)
    poly_B = poly_to_np(term2, n_rows)
    poly_C = poly_to_np(term3, n_rows)
    
    
    A = eval_poly(poly_A, g1_points)
    B = eval_poly(poly_B, g2_points)
    C = eval_poly(poly_C, g1_points)

    assert is_g1(A), "A is not in G1"
    assert is_g2(B), "B is not in G2"
    assert is_g1(C), "C is not in G1"

    left_side = pairing(B, A)

    g1s = [multiply(G1, int(tao**i) * int(t(tao)) ) for i in range(n_rows)]
    H_eval = eval_poly(poly_h, g1s)
    right_side = pairing(G2, add(C, H_eval))
    
    assert eq(left_side, right_side), "Left side is not equal to right side"
    return True


# Call the function
F_A = GF(mat_to_finite_field(A))
F_B = GF(mat_to_finite_field(B))
F_C = GF(mat_to_finite_field(C))
F_W = GF(mat_to_finite_field(w))

qap_finite_field(F_A, F_B, F_C, F_W)




TypeError: 'list_reverseiterator' object is not subscriptable